In [1]:
import sys
import os
import tensorflow
import tensorflow.keras as keras

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator #permite manipulacion de imágenes
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.keras import backend as K # De existir subprocesos en ejecución los finalizamos con la siguiente instrucción

K.clear_session() #cerramos los procesos en ejecución

datos_entrenamiento = './ClasificaImagenes/CarneDataset/train'
datos_prueba = '/Users/christianvega/Desktop/ClasificaImagenes/CarneDataset/test'

In [3]:
## Parametros

# 'iterac = 20' Numero de veces que vamos a iterar sobre nuestros datos de entrenamientos
iterac = 20

# Ajustamos el tamaño de las imagenes
altura, longitud = 150, 150

# Cantidad de Imagenes
cantidad_datos = 1640

# 'pasos = 1000' Numero de veces que se va a procesar la información en cada una de las iteraciones
pasos = 1000

pasos_validacion = 200

filtros_conv1 = 32
filtros_conv2 = 64

tamano_filtro1 = (3,3)
tamano_filtro2 = (2,2)

tamano_pool = (2,2)
clases = 8

# Learning Rate 
learning_rate = 0.0005

In [5]:
## PreProcesamiento de Imagenes

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

print('entrenamiento_datagen', train_datagen)

validacion_datagen = ImageDataGenerator(
    rescale = 1./255
)

imagen_entrenamiento = entrenamiento_datagen.flow_from_directory(
    datos_entrenamiento,
    target_size = (altura, longitud),
    batch_size = cantidad_datos,
    class_mode = 'categorical'
)

print('imagen_entrenamiento', imagen_entrenamiento)

imagen_validacion = validacion_datagen.flow_from_directory(
    datos_prueba,
    target_size = (altura, longitud),
    batch_size = cantidad_datos,
    class_mode = 'categorical'
)

entrenamiento_datagen <keras.preprocessing.image.ImageDataGenerator object at 0x7fbd0fda2bb0>
Found 1634 images belonging to 8 classes.
imagen_entrenamiento <keras.preprocessing.image.DirectoryIterator object at 0x7fbd0fdd8d30>
Found 810 images belonging to 8 classes.


In [ ]:
## Creando la Red CNN

cnn = Sequential()

cnn.add(
    Convolution2D(
        filtros_conv1,
        tamano_filtro1,
        padding = 'same',
        input_shape = (altura, longitud, 3),
        activation = 'relu'
    )
)

In [ ]:
cnn.add(
    MaxPooling2D(pool_size = tamano_pool)
)

cnn.add(
    Convolution2D(
        filtros_conv2,
        tamano_filtro2,
        padding = 'same',
        activation = 'relu'
    )
)

In [ ]:
cnn.add(
    MaxPooling2D(pool_size = tamano_pool)
)

cnn.add(Flatten())

cnn.add(
    Dense(256, activation = 'relu')
)

cnn.add(
    Dropout(0.5)
)

cnn.add(
    Dense(clases, activation = 'softmax')
)

cnn.compile(
    loss='sparse_categorical_crossentropy',
    optimizer = optimizers.Adam(learning_rate=learning_rate), #segundo error
    metrics = ['accuracy']
)

In [ ]:
cnn.fit(
    imagen_entrenamiento,
    steps_per_epoch = pasos,
    epochs = iterac,
    validation_data = imagen_validacion,
    validation_steps = pasos_validacion
)

target_dir = './modelo/'

if not os.path.exists(target_dir):
    os.mkdir(target_dir)

cnn.save('./modelo/modelo.h5')
cnn.save_weights('./modelo/pesos.h5')